In [ ]:
import pymysql
from openbb import obb

# MySQL 연결 설정
def connect_to_mysql():
    connection = pymysql.connect(
        host='host',
        user='username',
        password='password',
        database='database'
    )
    return connection

# Ticker 목록 가져오기
def get_all_tickers(cursor):
    sql = "SELECT id, ticker FROM stock_info"
    cursor.execute(sql)
    return cursor.fetchall()  # [(id, ticker), ...]

# StockPrices 데이터 삽입 함수
def insert_stock_price(cursor, stock_info_id, trade_date, open_price, close_price, high_price, low_price, volume):
    sql = """
    INSERT INTO stock_prices (ticker_id, trade_date, open_price, close_price, high_price, low_price, volume)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
    open_price = VALUES(open_price),
    close_price = VALUES(close_price),
    high_price = VALUES(high_price),
    low_price = VALUES(low_price),
    volume = VALUES(volume)
    """
    cursor.execute(sql, (stock_info_id, trade_date, open_price, close_price, high_price, low_price, volume))

# OpenBB에서 특정 날짜의 가격 데이터 가져오기
def fetch_historical_price(ticker, start_date, end_date):
    df = obb.equity.price.historical(symbol=ticker, start_date=start_date, end_date=end_date)
    return df.to_df()  # Pandas DataFrame 반환

# Main 실행
if __name__ == "__main__":
    start_date = "2024-12-01"
    end_date = "2024-12-30"

    connection = None
    try:
        # MySQL 연결
        connection = connect_to_mysql()
        cursor = connection.cursor()

        # Step 1: stock_info에서 모든 티커 가져오기
        tickers = get_all_tickers(cursor)
        print(f"Found {len(tickers)} tickers in stock_info.")

        # Step 2: 각 티커에 대해 주가 데이터 가져오기 및 삽입
        for stock_info_id, ticker in tickers:
            try:
                print(f"Processing {ticker}...")

                # OpenBB를 이용해 주가 데이터 가져오기
                historical_price_df = fetch_historical_price(ticker, start_date=start_date, end_date=end_date)
                print(historical_price_df)

                # DataFrame을 사용하여 데이터를 stock_prices 테이블에 삽입
                for index, row in historical_price_df.iterrows():
                    trade_date = index.strftime('%Y-%m-%d')  # 날짜 형식 변환
                    insert_stock_price(
                        cursor,
                        stock_info_id,
                        trade_date,
                        row['open'],      # open_price
                        row['close'],     # close_price
                        row['high'],      # high_price
                        row['low'],       # low_price
                        row['volume']     # volume
                    )

                connection.commit() # 단건 커밋
                print(f"Price data for {ticker} inserted successfully.")
            except Exception as e:
                print(f"Error processing {ticker}: {e}")
                connection.rollback()

    except pymysql.MySQLError as e:
        print(f"MySQL Error: {e}")
    finally:
        if connection:
            connection.close()


In [ ]:
import pymysql
import logging
from openbb import obb

# MySQL 연결 설정
def connect_to_mysql():
    connection = pymysql.connect(
        host='host',
        user='username',
        password='password',
        database='database'
    )
    return connection

# Ticker 목록 가져오기
def get_all_tickers(cursor):
    sql = "SELECT id, ticker FROM stock_info"
    cursor.execute(sql)
    return cursor.fetchall()  # [(id, ticker), ...]

# StockPrices 데이터 삽입 함수
def insert_stock_price(cursor, stock_info_id, trade_date, open_price, close_price, high_price, low_price, volume):
    sql = """
    INSERT IGNORE INTO stock_prices (ticker_id, trade_date, open_price, close_price, high_price, low_price, volume)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(sql, (stock_info_id, trade_date, open_price, close_price, high_price, low_price, volume))

# OpenBB에서 특정 날짜의 가격 데이터 가져오기
def fetch_historical_price(ticker, start_date, end_date):
    df = obb.equity.price.historical(symbol=ticker, start_date=start_date, end_date=end_date)
    return df.to_df()  # Pandas DataFrame 반환

# 로깅 설정
logging.basicConfig(
    filename="stock_price_update.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# Main 실행
if __name__ == "__main__":
    start_date = "2024-12-01"
    end_date = "2024-12-03"
    batch_size = 5  # Batch 크기 설정

    connection = None
    try:
        # MySQL 연결
        connection = connect_to_mysql()
        cursor = connection.cursor()

        # Step 1: stock_info에서 모든 티커 가져오기
        tickers = get_all_tickers(cursor)
        logging.info(f"Found {len(tickers)} tickers in stock_info.")

        # Step 2: Batch 처리로 각 티커에 대해 주가 데이터 가져오기 및 삽입
        for batch_start in range(0, len(tickers), batch_size):
            batch = tickers[batch_start:batch_start + batch_size]
            logging.info(f"Processing batch {batch_start // batch_size + 1}: {len(batch)} tickers")

            for stock_info_id, ticker in batch:
                try:
                    logging.info(f"Processing {ticker}...")

                    # OpenBB를 이용해 주가 데이터 가져오기
                    historical_price_df = fetch_historical_price(ticker, start_date=start_date, end_date=end_date)
                    logging.info(f"Fetched data for {ticker}. Rows: {len(historical_price_df)}")

                    # DataFrame을 사용하여 데이터를 stock_prices 테이블에 삽입
                    for index, row in historical_price_df.iterrows():
                        trade_date = index.strftime('%Y-%m-%d')  # 날짜 형식 변환

                        # 데이터 삽입
                        insert_stock_price(
                            cursor,
                            stock_info_id,
                            trade_date,
                            row['open'],      # open_price
                            row['close'],     # close_price
                            row['high'],      # high_price
                            row['low'],       # low_price
                            row['volume']     # volume
                        )

                    connection.commit()
                    logging.info(f"Price data for {ticker} inserted successfully.")
                except Exception as e:
                    logging.error(f"Error processing {ticker}: {e}")
                    connection.rollback()  # 해당 티커만 롤백 처리

    except pymysql.MySQLError as e:
        logging.critical(f"MySQL Error: {e}")
    finally:
        if connection:
            connection.close()
        logging.info("Database connection closed.")

In [ ]:
import asyncio
import nest_asyncio
import aiomysql
import logging
from openbb import obb

nest_asyncio.apply()

# 로깅 설정 (파일에 저장)
logging.basicConfig(
    level=logging.INFO,  # 로그 레벨 설정
    format="%(asctime)s - %(levelname)s - %(message)s",  # 로그 포맷
    handlers=[
        logging.FileHandler("stock_price_update.log"),  # 로그를 파일에 저장
        #logging.StreamHandler()  # 로그를 터미널에도 출력
    ]
)

# MySQL 연결 설정
async def connect_to_mysql():
    return await aiomysql.connect(
        host='host',
        user='username',
        password='password',
        database='database',
        minsize=1,
        maxsize=40  # 동시 연결 최대 개수
    )

# Ticker 목록 가져오기
async def get_all_tickers(cursor):
    sql = "SELECT id, ticker FROM stock_info"
    await cursor.execute(sql)
    return await cursor.fetchall()  # [(id, ticker), ...]

# Batch Insert 함수
async def batch_insert_stock_prices(cursor, batch_data):
    sql = """
    INSERT IGNORE INTO stock_prices (ticker_id, trade_date, open_price, close_price, high_price, low_price, volume)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    await cursor.executemany(sql, batch_data)

# Ticker 데이터 처리
async def process_ticker(pool, stock_info_id, ticker, start_date, end_date, semaphore):
    async with semaphore:
        async with pool.acquire() as conn:
            async with conn.cursor() as cursor:
                try:
                    df = obb.equity.price.historical(symbol=ticker, start_date=start_date, end_date=end_date).to_df()
                    df = df.dropna(subset=['open', 'close', 'high', 'low', 'volume'])
                    df = df[(df['open'] > 0) & (df['close'] > 0) & (df['high'] > 0) & (df['low'] > 0)]

                    batch_data = [
                        (
                            stock_info_id,
                            index.strftime('%Y-%m-%d'),
                            row['open'],
                            row['close'],
                            row['high'],
                            row['low'],
                            row['volume']
                        )
                        for index, row in df.iterrows()
                    ]

                    batch_size = 500  # 하나의 티커에 대한 데이터들을 배치 단위로 처리
                    for i in range(0, len(batch_data), batch_size):
                        batch = batch_data[i:i + batch_size]  # 배치 데이터 슬라이싱
                        await batch_insert_stock_prices(cursor, batch)  # 배치 단위 삽입

                    await conn.commit()  # 티커 단위 커밋
                    logging.info(f"Processed {ticker}: {len(batch_data)} rows inserted.")
                except Exception as e:
                    logging.error(f"Error processing {ticker}: {e}")
                    await conn.rollback()

# 메인 함수
async def main():
    start_date = "2022-01-01"
    end_date = "2022-12-31"
    pool = await aiomysql.create_pool(
        host='host',
        user='username',
        password='password',
        database='database',
        minsize=1,
        maxsize=40  # MySQL 연결 풀
    )

    semaphore = asyncio.Semaphore(60)  # 병렬 작업 제한

    async with pool.acquire() as conn:
        async with conn.cursor() as cursor:
            tickers = await get_all_tickers(cursor)

    tasks = [process_ticker(pool, stock_info_id, ticker, start_date, end_date, semaphore) for stock_info_id, ticker in tickers]
    await asyncio.gather(*tasks)  # 비동기로 모든 티커 처리

    pool.close()
    await pool.wait_closed()

# 실행
await main()